In [91]:
import pandas as pd
import os
import openpyxl
import numpy as np
import re
from datetime import datetime

In [2]:
class FinancialReport():
    def __init__(self,file):
        self.file = file
        self.readFile()
        self.findBalanceSheet()
    def readFile(self):
        path = os.path.join("financial statements",self.file)
        self.statements = pd.read_excel(path,sheet_name=None)
    def getSheetNames(self):
        return self.statements.keys()
    def findBalanceSheet(self):
        balanceSheetName = [name for name in self.getSheetNames() if "BALANCE" in name][0]
        self.balance = self.statements[balanceSheetName]
        self.balance.columns = ["labels"]+list(self.balance.columns[1:])
    def getBalanceSheet(self):
        return self.balance
    

In [113]:
class Balance():
    def __init__(self):
        self.finalBalance = pd.DataFrame(columns=["title","num_title","labels"])
        self.setFinancialStatements()
        self.addDataToBalanceSheet()
        self.rearrangeFinalBalanceSheet()
        self.setQuarters()
        
    def getFiles(self):
        files = os.listdir("financial statements")
        files.remove(".DS_Store")
        files = [file for file in files if file[0] != "~"]
        return files

    def setFinancialStatements(self):
        self.files = self.getFiles()
        self.FinancialReports = [FinancialReport(file) for file in self.files]
        
    def addDataToBalanceSheet(self):
        
        for FinancialStatements in self.FinancialReports:
            balance = FinancialStatements.getBalanceSheet().copy()
            
            cols = ["labels"]+[col for col in balance.columns[1:] if col not in self.finalBalance.columns] ## remove columns of data already in main balance sheet
            balance = balance[cols]
            
            balance = self.parse_attributes(balance)
            
            balance.loc[:,"labels"] = [re.sub(" \$|\s*\d+\,*", "", label)  if np.any([char.isdigit() for char in label]) else label for label in balance["labels"]  ]
            balance.labels = balance.labels.apply(self.clean_up_string)
            
            
            self.finalBalance = pd.merge(self.finalBalance, balance, how="outer",on=["labels"],suffixes=('', '_new'))
            self.finalBalance.title = [self.finalBalance.title[i] if not pd.isnull(self.finalBalance.title[i]) else self.finalBalance.title_new[i] for i in range(len(self.finalBalance))] #merge titles
            self.finalBalance.num_title = [self.merge_number(self.finalBalance.num_title[i],self.finalBalance.num_title_new[i]) for i in range(len(self.finalBalance))] #merge the number for the titles. use greatest num bc bigger num means there is section before it
            self.finalBalance = self.finalBalance.drop(["title_new","num_title_new"],axis=1)

    def rearrangeFinalBalanceSheet(self):
        self.finalBalance = self.finalBalance.replace(" ",np.nan)

        # put totals at the bottom of each section
        totals = self.finalBalance.loc[self.finalBalance.labels.str.startswith("Total")].reset_index(drop=True) 
        self.finalBalance = self.finalBalance.loc[~self.finalBalance.labels.str.startswith("Total")].reset_index(drop=True)
        self.finalBalance = pd.concat([self.finalBalance,totals]) 
        self.finalBalance = self.finalBalance.reset_index(drop=True).reset_index()
        self.finalBalance = self.finalBalance.sort_values(["num_title","index"]) # order by title and then location
        self.finalBalance = self.finalBalance.drop(["index","num_title"],axis=1)

    def parse_attributes(self,balance):
        num = 0
        index = []
        for label in balance.labels:
            if ":" in label:
                title = label[:-1]
                num += 1
                continue
            index.append([title,num,label])
            if "Total" in label:
                title = np.nan
                num += 1

        balance = balance.loc[balance.labels.isin(np.array(index)[:,2])].copy()
        balance.loc[:,["title","num_title","labels"]] = list(index)
        balance = balance.set_index(["title","num_title","labels"]).reset_index()
        return balance
    
    def clean_up_string(self,string):
        string = string.strip()
        space_positions = [x.start() for x in re.finditer('\ ',  string)]
        for i in reversed(space_positions):
            if string[i+1:] in ["of", "and"]:
                string = string[:i]
        return string
    
    def merge_number(self,a,b):
        if pd.isnull(a):
            return b
        if pd.isnull(b):
            return a
        return max(a,b)
    def setQuarters(self):
        dates = list(self.finalBalance.columns[2:])
        dates.sort(key=lambda date: datetime.strptime(date.replace(".",""), '%b %d, %Y'))
        self.finalBalance = self.finalBalance[["title","labels"]+dates]
        self.finalBalance.columns = ["title","labels"]+[self.parseQuarter(date) for date in self.finalBalance.columns[2:]]
    def parseQuarter(self,date):
        mo = date.split(" ")[0].strip(".")
        year = date.split(", ")[-1]
        if mo in ["May"]:
            return "Q1 "+str(int(year)+1)
        
        if mo in ["Jul","Aug"]:
            return "Q2 "+str(int(year)+1)
        
        if mo in ["Oct","Nov"]:
            return "Q3 "+str(int(year)+1)
        
        if mo in ["Jan","Feb"]:
            return "Q4 "+str(int(year))
        
        

In [116]:
balance = Balance()
balance.finalBalance

,title,labels,Q4 2018,Q1 2019,Q2 2019,Q3 2019,Q4 2019,Q1 2020,Q2 2020,Q3 2020,Q4 2020,Q1 2021,Q2 2021,Q3 2021,Q4 2021,Q1 2022,Q2 2022,Q3 2022,Q4 2022
0,Current assets,Cash and cash equivalents,1783.0,1210.0,1322.0,958.0,1081.0,941.0,1177.0,788.0,1364.0,1028.0,2188.0,2471.0,1988.0,2066.0,2375.0,801.0,877.00
1,Current assets,Short-term investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178.0,410.0,NaN,NaN,275.0,0.00
2,Current assets,Merchandise inventory,1997.0,2035.0,2202.0,2668.0,2131.0,2242.0,2326.0,2720.0,2156.0,2217.0,2242.0,2747.0,2451.0,2370.0,2281.0,2721.0,3018.00
3,Current assets,Other current assets,788.0,778.0,780.0,792.0,751.0,757.0,770.0,770.0,706.0,920.0,882.0,966.0,1159.0,1091.0,1201.0,1410.0,1270.00
23,Current assets,"Available-for-sale Securities, Current",NaN,164.0,286.0,296.0,288.0,272.0,294.0,294.0,290.0,51.0,25.0,NaN,NaN,475.0,337.0,NaN,NaN
25,Current assets,Short-term Investments,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,Current assets,"Cash and Cash Equivalents, at Carrying Value",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,Current assets,"Other Assets, Current",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,Current assets,Total current assets,4568.0,4187.0,4590.0,4714.0,4251.0,4212.0,4567.0,4572.0,4516.0,4216.0,5337.0,6362.0,6008.0,6002.0,6194.0,5207.0,5165.00
4,NaN,"Property and equipment, net of accumulated dep...",2805.0,2791.0,2832.0,2887.0,2912.0,3129.0,3141.0,3225.0,3122.0,2945.0,2895.0,2846.0,2841.0,2839.0,2897.0,2924.0,3037.00


In [117]:
balance.finalBalance.to_csv("finalBalance.csv")

In [79]:
'Oct. 30, 2021'.split(", ")[-1]

'2021'